<a href="https://colab.research.google.com/github/Cherishma19-samala/HDS5210-02-Assignments/blob/main/week12_assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 11 Assignment


Please do the programming exercise and verify that your code works using the tests, then think about your final project and fill out the questions in the second part.

---
---

### 47.1: Filtering and summarizing data

For this work, you'll find a data file in `https://hds5210-data.s3.amazonaws.com/complications_all.csv`.

Read in the data file and create a variable called `mo_hospitals` that contains a data frame from the `complications_all.csv` file, filtered down to only contain those hospitals from the state of Missouri (MO).

Then aggregate that data by hospital into a variable named `mo_summary`.  There are some key fields that we want to summarize:
* We want to know the earliest date that each hospital was participating in any program
* We want to know the latest date that each hospital stopped participating in any program
* We want to know the total number of patients in the denominators of these programs

Some things to note:
* You will need to convert the `Start Date` and `End Date` to actual datetime fields
* You will need to clean up and convert the `Denominator` field to just be numeric - the rule that you should use it to simply remove any records where the `Denominator` is `'Not Available'`


The final result of this step should be a new data frame called `mo_summary` that contains one row for each hospital and contains the min start date, max end date, and total denominator.  Use the names `start_date`, `end_date`, and `number` for those columns in `mo_summary`.


You do not need to create your code in the form of a function, just make sure your variable names match what I've described above so the tests work.

In [1]:
import pandas as pd
# This is just to show you the name to use for the variable you need to create for this step to pass.
all_hospitals = pd.read_csv('https://hds5210-data.s3.amazonaws.com/complications_all.csv')


In [2]:
# Do you work here and in as many cells as you need to create a variable called `mo_summary` that matches the requirements

In [3]:
all_hospitals.head()

,Facility ID,Facility Name,Address,City,State,ZIP Code,County Name,Phone Number,Measure ID,Measure Name,Compared to National,Denominator,Score,Lower Estimate,Higher Estimate,Footnote,Start Date,End Date
0,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,COMP_HIP_KNEE,Rate of complications for hip/knee replacement...,No Different Than the National Rate,292,3.2,2.1,4.8,NaN,04/01/2015,03/31/2018
1,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,MORT_30_AMI,Death rate for heart attack patients,No Different Than the National Rate,688,13,11.0,15.5,NaN,07/01/2015,06/30/2018
2,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,MORT_30_CABG,Death rate for CABG surgery patients,No Different Than the National Rate,291,4.3,2.6,6.8,NaN,07/01/2015,06/30/2018
3,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,MORT_30_COPD,Death rate for COPD patients,No Different Than the National Rate,411,8.8,6.7,11.4,NaN,07/01/2015,06/30/2018
4,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,MORT_30_HF,Death rate for heart failure patients,No Different Than the National Rate,869,12.7,10.7,15.0,NaN,07/01/2015,06/30/2018


In [4]:
MO = all_hospitals['State'] == 'MO'
ho = all_hospitals[MO]
final = ho[['Facility Name','Start Date', 'End Date', 'Denominator']]


In [5]:
# Converting and Assigning
final = final.copy()

start_date = final['Start Date'].astype(str)
end_date = final['End Date'].astype(str)

start_date = pd.to_datetime(start_date.str[0:10], format="%m/%d/%Y")
end_date = pd.to_datetime(end_date.str[0:10], format="%m/%d/%Y")
final['start_date'] = start_date
final['end_date'] = end_date

In [6]:
# Filtering, Convert and Assign
d = final['Denominator'] != 'Not Available'
r = final[d].copy()

r['number'] = pd.to_numeric(r['Denominator'])
r['hospital'] = r['Facility Name']

r.head()

,Facility Name,Start Date,End Date,Denominator,start_date,end_date,number,hospital
45534,MERCY HOSPITAL JOPLIN,04/01/2015,03/31/2018,26,2015-04-01,2018-03-31,26,MERCY HOSPITAL JOPLIN
45535,MERCY HOSPITAL JOPLIN,07/01/2015,06/30/2018,175,2015-07-01,2018-06-30,175,MERCY HOSPITAL JOPLIN
45536,MERCY HOSPITAL JOPLIN,07/01/2015,06/30/2018,91,2015-07-01,2018-06-30,91,MERCY HOSPITAL JOPLIN
45537,MERCY HOSPITAL JOPLIN,07/01/2015,06/30/2018,326,2015-07-01,2018-06-30,326,MERCY HOSPITAL JOPLIN
45538,MERCY HOSPITAL JOPLIN,07/01/2015,06/30/2018,461,2015-07-01,2018-06-30,461,MERCY HOSPITAL JOPLIN


In [7]:
# Drop
r = r.drop(columns=['Denominator'])

# Group by and aggregate
mo_summary = r.groupby('hospital', as_index=False)['number'].sum()

# Find min start
start_date = r.groupby('hospital', as_index=False)['start_date'].min()

# Find max end
end_date = r.groupby('hospital', as_index=False)['end_date'].max()

# Merge the results
mo_summary = pd.merge(mo_summary, start_date, on='hospital')
mo_summary = pd.merge(mo_summary, end_date, on='hospital')

In [8]:
mo_summary.set_index('hospital', inplace=True)

In [9]:
mo_summary


,number,start_date,end_date
hospital,,,
BARNES JEWISH HOSPITAL,131313,2015-04-01,2018-06-30
BARNES-JEWISH ST PETERS HOSPITAL,15668,2015-04-01,2018-06-30
BARNES-JEWISH WEST COUNTY HOSPITAL,9622,2015-04-01,2018-06-30
BATES COUNTY MEMORIAL HOSPITAL,3117,2015-07-01,2018-06-30
BELTON REGIONAL MEDICAL CENTER,9270,2015-04-01,2018-06-30
...,...,...,...
TRUMAN MEDICAL CENTER LAKEWOOD,4297,2015-04-01,2018-06-30
UNIVERSITY OF MISSOURI HEALTH CARE,56493,2015-04-01,2018-06-30
WASHINGTON COUNTY MEMORIAL HOSPITAL,220,2015-07-01,2018-06-30


In [10]:
assert(mo_summary['number'].sum() == 1766908)
assert(mo_summary['start_date'].min() == pd.Timestamp(2015,4,1))
assert(mo_summary['end_date'].max() == pd.Timestamp(2018,6,30))
assert(mo_summary.shape == (108,3))
assert(mo_summary.loc['BARNES JEWISH HOSPITAL'].number == 131313)
assert(mo_summary.loc['BOONE HOSPITAL CENTER'].number == 63099)

---

### 47.2 Planning your final project

You should be thinking about the things we've been learning and how you can apply them to your final project.  Use the rubric to help guid your thinking and then answer the questions below.  This is meant as a guide to help you think through what you will do.

#### A) Data Access

Your project should include data from at least three distinct types of sources.  For example: AWS S3, Relational Databases, Internet, Web Services, local files.  List what data sources you're planning to use.

**Double-click to enter your answer**

Kaggle, Hugging face, Data.world


#### B. Data Formats

Your project should include data that comes in different file formats.  For example: HL7, EDI, HTML, CSV, Excel, JSON, XML.  List what data formats you're planning to use.

**Double-click to enter your answer**

TXT, CSV, XLSX


#### C. Objective

What purpose would your project serve in a real work setting?  Take a couple of paragraphs to write down why this is an interesting product.

**Double-click to enter your answer**

I propose to identify key risk factors in order to reduce short term mortality and enhance survival in heart failure patients and thereby their quality of life. Thus identifying these factors the project can inform specific directions for healthcare prevention and improvement efforts to manage risk factors and improve patients’ health.




---



## Submit your work via GitHub as normal
